# HW 5: COMET-ATOMIC Schema

In this homework, you will create your own schema to represent the state of a story world as it goes through the story line by line.
A **schema** is a structured reprensentation made to hold facts or a plan, which in this case, can be used to track change over time.

**The purpose of this homework is to test your understanding of schemas and get hands-on experience with a state-of-the-art tool in commonsense reasoning.**


## Your Task
You will be creating a schema using ATOMIC to track the state of a fictional world. For each sentence of the story, you will parse it (provided), call COMET (provided, but what you input is up to you), create preconditions to determine if a sentence can be added (TODO), and create effects to use to update your schema (TODO).

Let's teach your agent some basic information about the world!

------------------------

Formally, the task is:

Given an input sentence at time *t* (*In_t*), produce a schema *S_t*. Do this for each sentence in the story.

For example, using VerbNet:

| Timestep | Input | Schema |
|--------|-------|--------|
| 1 | Bethany picks up the sword. | `Bethany: has_possesion(sword)` |
| 2 | Bethany throws the sword. | `Bethany: !has_possesion(sword)` |

*In_1* - "Bethany picks up the sword."

would produce

*S_1* - `Bethany: has_possesion(sword)`

But then if the next sentence is:

*In_2* - "Bethany throws the sword."

The state would be updated to

*S_2* - `Bethany: !has_possesion(sword)`

-----------------------------
Your resulting system will be sort of like this simplified diagram (the parser is provided for you):
![Given a sentence_t and the knowledge representation from your knowledge database of choice, produce schemas (via some schema processor you create)](https://interactive-fiction-class.org/homeworks/schemas/schemas.png)

There is some knowledge about the story world, and you are using a schema to feed this information into bite-sized chunks so that your agent (and you) can understand it.
You will then have a processing step on the schema where you update it as you get more information as the story progresses.


To reiterate, you will:
1. Get to know [COMET-ATOMIC-2020](https://aaai.org/ojs/index.php/AAAI/article/view/4160). (Alternate link in case AAAI is down: https://arxiv.org/pdf/2010.05953.pdf)
2. Make a __schema manipulator__ (not a real term, but I think it sounds cool), which will take in knowledge from ATOMIC and spit out your schema. Skeleton code is provided for you, but you are welcome to change things so that it makes more sense to you. This involves two steps:

  a. Validate: Generate preconditions to determine if an event can be added.

  b. Update: Generate effects to update your world state.  



# What is ATOMIC?

ATOMIC is a commonsense knowledge graph with some social inferences, among other things.

```
@inproceedings{sap2019atomic,
   title={ATOMIC: An Atlas of Machine Commonsense for If-Then Reasoning},
   author={Sap, Maarten and LeBras, Ronan and Allaway, Emily and Bhagavatula, Chandra and Lourie, Nicholas and Rashkin, Hannah and Roof, Brendan and Smith, Noah A and Choi, Yejin},
   year={2019},
   booktitle={AAAI},
   url={https://aaai.org/ojs/index.php/AAAI/article/view/4160}
}
```

It contains the following inferences about people and events:

* Because PersonX wanted (xIntent)
* Before, PersonX needed (xNeed, HasPrerequisite)
* PersonX is seen as (xAttr)
* As a result, PersonX feels (xReact)
* As a result, PersonX wants (xWant)
* As a result, PersonX reasons (xReason)
* PersonX then (xEffect)
* As a result, others feel (oReact)
* As a result, others want (oWant)
* Others then (oEffect)
* Happens before (isBefore)
* Happens after (isAfter)
* Is hindered by (HinderedBy)
* Causes (Causes)

and inferences about entities:
* Is located at (AtLocation, LocatedNear, LocationOfAction)
* Is made up of (MadeUpOf, PartOf. NotMadeOf)
* Is used to (UsedFor, ObjectUse)
* Has the property (HasProperty, NotHasProperty)
* Is capable of (CapableOf, NotCapableOf)
* Desires (Desires, NotDesires)


Among other things (full list is in code block 41).
Or you can play with it online here: https://mosaickg.apps.allenai.org/kg_atomic2020
https://mosaickg.apps.allenai.org/model_comet2020_people_events
https://mosaickg.apps.allenai.org/model_comet2020_entities


[COMET](https://aclanthology.org/P19-1470) is the model that is trained on ATOMIC.

#Setup

## Get COMET and Install packages

Repo: https://github.com/allenai/comet-atomic-2020/

In [1]:
# Clone the repo
%%capture
!git clone https://github.com/allenai/comet-atomic-2020.git

In [2]:
# Enter the directory
import os
os.chdir('comet-atomic-2020')

In [3]:
# Install ATOMIC's dependencies
%%capture
!pip install rouge_score
!pip install transformers
!pip install -r requirements.txt

### Also, Stanford's Stanza Parser

In order to process the input sentences, you will need to do some parsing. Here, we have setup Stanford's English language NER (Named Entity Recognition) and constituency parser using [Stanza](https://stanfordnlp.github.io/stanza/index.html). You're also welcome to use any of the other parsers they provide (or even switch to a completely different type of parser that you like better).

In [4]:
%%capture
!python -m pip install stanza

import stanza
stanza.download('en')

**You might need to restart the runtime now.**


In [5]:
# Example code to run Stanza

import stanza #here's the re-import for when your runtime is restarted
import json
nlp = stanza.Pipeline('en', processors='tokenize, ner, mwt') #lemma, depparse, constituency, pos
parse = nlp("Aurora submitted her resignation to Facebook.")
y = json.loads(str(parse))
y

[[{'end_char': 6, 'id': 1, 'ner': 'S-ORG', 'start_char': 0, 'text': 'Aurora'},
  {'end_char': 16, 'id': 2, 'ner': 'O', 'start_char': 7, 'text': 'submitted'},
  {'end_char': 20, 'id': 3, 'ner': 'O', 'start_char': 17, 'text': 'her'},
  {'end_char': 32,
   'id': 4,
   'ner': 'O',
   'start_char': 21,
   'text': 'resignation'},
  {'end_char': 35, 'id': 5, 'ner': 'O', 'start_char': 33, 'text': 'to'},
  {'end_char': 44,
   'id': 6,
   'ner': 'S-ORG',
   'start_char': 36,
   'text': 'Facebook'},
  {'end_char': 45, 'id': 7, 'ner': 'O', 'start_char': 44, 'text': '.'}]]

## COMET-ATOMIC-2020 (BART) Setup

In [6]:
# Download the model
%%capture
!bash models/comet_atomic2020_bart/download_model.sh

**Tip: Take note of the `all_relations` dictionary below! You will need it later on.**

In [7]:
# Load the model
# copied from models/comet_atomic2020_bart/generation_example.py

import json
import torch
import argparse
from tqdm import tqdm
from pathlib import Path
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from models.comet_atomic2020_bart.utils import calculate_rouge, use_task_specific_params, calculate_bleu_score, trim_batch


def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i : i + n]


class Comet:
    def __init__(self, model_path):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(self.device)
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        task = "summarization"
        use_task_specific_params(self.model, task)
        self.batch_size = 1
        self.decoder_start_token_id = None

    def generate(
            self, 
            queries,
            decode_method="beam", 
            num_generate=5, 
            ):

        with torch.no_grad():
            examples = queries

            decs = []
            for batch in list(chunks(examples, self.batch_size)):

                batch = self.tokenizer(batch, return_tensors="pt", truncation=True, padding="max_length").to(self.device)
                input_ids, attention_mask = trim_batch(**batch, pad_token_id=self.tokenizer.pad_token_id)

                summaries = self.model.generate(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    decoder_start_token_id=self.decoder_start_token_id,
                    num_beams=num_generate,
                    num_return_sequences=num_generate,
                    )

                dec = self.tokenizer.batch_decode(summaries, skip_special_tokens=True, clean_up_tokenization_spaces=False)
                decs.append(dec)

            return decs


all_relations = [
    "AtLocation",
    "CapableOf",
    "Causes",
    "CausesDesire",
    "CreatedBy",
    "DefinedAs",
    "DesireOf",
    "Desires",
    "HasA",
    "HasFirstSubevent",
    "HasLastSubevent",
    "HasPainCharacter",
    "HasPainIntensity",
    "HasPrerequisite",
    "HasProperty",
    "HasSubEvent",
    "HasSubevent",
    "HinderedBy",
    "InheritsFrom",
    "InstanceOf",
    "IsA",
    "LocatedNear",
    "LocationOfAction",
    "MadeOf",
    "MadeUpOf",
    "MotivatedByGoal",
    "NotCapableOf",
    "NotDesires",
    "NotHasA",
    "NotHasProperty",
    "NotIsA",
    "NotMadeOf",
    "ObjectUse",
    "PartOf",
    "ReceivesAction",
    "RelatedTo",
    "SymbolOf",
    "UsedFor",
    "isAfter",
    "isBefore",
    "isFilledBy",
    "oEffect",
    "oReact",
    "oWant",
    "xAttr",
    "xEffect",
    "xIntent",
    "xNeed",
    "xReact",
    "xReason",
    "xWant",
    ]

print("model loading ...")
comet = Comet("./comet-atomic_2020_BART")
comet.model.zero_grad()
print("model loaded")



model loading ...
model loaded


Run your queries. `head` is the input sentence, and `rel` is the relation, as seen in `all_relations`.

In [8]:
# Example code to run COMET. A method has been written for you further down called callCOMET().
queries = []
head = "PersonX relies on PersonY"
rel = "xNeed"
query = "{} {} [GEN]".format(head, rel)
queries.append(query)
print(queries)
results = comet.generate(queries, decode_method="beam", num_generate=5)
print(results)

['PersonX relies on PersonY xNeed [GEN]']
[[' to ask PersonY for help', ' to be dependent on PersonY', ' to ask PersonY to help', ' to know PersonY', ' to ask for help']]


In [9]:
"""
# Optional: Get ATOMIC data (if you wanted the train/test/val sets)
!wget https://ai2-atomic.s3-us-west-2.amazonaws.com/data/atomic2020_data-feb2021.zip
!unzip atomic2020_data-feb2021.zip
"""

'\n# Optional: Get ATOMIC data (if you wanted the train/test/val sets)\n!wget https://ai2-atomic.s3-us-west-2.amazonaws.com/data/atomic2020_data-feb2021.zip\n!unzip atomic2020_data-feb2021.zip\n'

# Parse the sentence to feed into COMET
You're welcome to change this to fit your needs

In [10]:
import nltk
from nltk.tree import Tree
from collections import Counter, defaultdict


class SentParser:
  """
  Parse the sentence and get the entities and the new phrase with tags
  """
  def __init__(self, sentence):
    sentence = sentence.replace(".","")
    parse = nlp(sentence) # call Stanza
    self.phrase = sentence # original sentence
    self.entities = dict() # dict of labels for entities e.g., [{PersonX: John}]
    self.new_phrase = sentence # the sentence with person names changed to PersonX and PersonY
    
    for sentence in parse.sentences:
      ents = self.getEntities(sentence.tokens)
      self.entities = ents

      for tag in ents.keys():
        person = ents[tag]
        self.new_phrase = self.new_phrase.replace(person, tag)
        
  def getEntities(self, parse):
    """
    get the named entities so you can pass it to ATOMIC's input and
    fill the PersonX and PersonY tags from the output

    args:
    parse (list) - list of Stanza token objects for this phrase

    return:
    entities (dict) - keeps track of who is PersonX and PersonY e.g. {PersonX: John}
    """
    entities = dict()
    count = 0
    for word in parse:
      if "PERSON" in word.ner:
        if count == 0:
          entities['PersonX'] = word.text 
          count+=1
        elif count == 1:
          entities['PersonY'] = word.text
    return entities


In [11]:
# Example call
s = SentParser("John went to the bank.")
print(s.phrase)
print(s.entities)
print(s.new_phrase)

John went to the bank
{'PersonX': 'John'}
PersonX went to the bank


In [12]:
s = SentParser("John and Mary went to the bank.")
print(s.phrase)
print(s.entities)
print(s.new_phrase)

John and Mary went to the bank
{'PersonX': 'John', 'PersonY': 'Mary'}
PersonX and PersonY went to the bank


# TODO: Setup your schema

You're welcome to use all or just a subset of the relations from `all_relations`. Work with whatever you think makes sense.

Tip: You might want to not take every single fact that ATOMIC gives you. Try to come up with a heuristic to just take what you need.

In [13]:
preq_relations = [
    "AtLocation",
    "CapableOf",
    "CreatedBy",
    "DefinedAs",
    "DesireOf",
    "HasA",
    "HasFirstSubevent",
    "HasLastSubevent",
    "HasPainCharacter",
    "HasPainIntensity",
    "HasPrerequisite",
    "HasProperty",
    "HasSubEvent",
    "HasSubevent",
    "HinderedBy",
    "InheritsFrom",
    "InstanceOf",
    "IsA",
    "LocatedNear",
    "LocationOfAction",
    "MadeOf",
    "MadeUpOf",
    "MotivatedByGoal",
    "NotCapableOf",
    "NotDesires",
    "NotHasA",
    "NotHasProperty",
    "NotIsA",
    "NotMadeOf",
    "PartOf",
    "RelatedTo",
    "SymbolOf",
    "UsedFor",
    "isAfter",
    "isBefore",
    "isFilledBy",
    "xIntent",
    "xNeed",
    "oWant",
    ]
effects_relations = [
    "Causes",
    "CausesDesire",
    "Desires",
    "oEffect",
    "oReact",
    "xAttr",
    "xEffect",
    "xReact",
    "xReason",
    "xWant",
  ]

In [14]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 827.9 MB 1.1 MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180942 sha256=0c05b75c89055c38224ef3a957c8c839708264e086b8078ee188692868dbae1b
  Stored in directory: /tmp/pip-ephem-wheel-cache-osskbhw0/wheels/11/95/ba/2c36cc368c0bd339b44a791c2c1881a1fb714b78c29a4cb8f5
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [15]:
import spacy
import en_core_web_lg
nlpp = en_core_web_lg.load()

In [16]:
def callCOMET(sent, rel, decode="beam", num=5):
  """
  Making COMET generate facts based on an input sent and a relation rel
  You can also provide the decoding method and the number of facts 
  you want it to output.
  """
  query = ["{} {} [GEN]".format(sent, rel)]
  gen = comet.generate(query, decode_method=decode, num_generate=num)[0]
  return [s.strip().replace(".","") for s in gen if s.strip() != "none" or s.strip() != "."]

def fillEntityTags(fact, NER):
  """
  Given a output fact from COMET (str) and the NER (dict), replace the PersonX/Y
  tags with their original names
  """
  new = fact
  for entity in NER.keys():
    new = new.replace(entity,NER[entity])
  return new

class Predicate:
  """
  Individual precondition and effect objects
  """
  def __init__(self, rel, statement, isPre, neg=False):
    self.relation = rel # string - input COMET relation
    self.statement = statement # string - results from COMET
    self.isPrecondition = isPre # boolean - if it's a precondition (True) or an effect (False)
    self.isNegated = neg # boolean - if it's negated (True) or not (False)

class Schema:
  """
  Your schema
  """
  def __init__(self, starting_state):
    self.state = starting_state # defaultdict(set) - a set of facts for each entity
    self.curr_event = "" # string - received from the parser; phrase from the original input sentence with PersonX/PersonY labels
    self.curr_NER = dict() # dict - person names and their corresponding tags for a given subevent/phrase
    self.preconditions = [] # list - stores Predicate objects
    self.timestep = 0 # int - how far you are in the story (optional)

  ### Check the preconditions against the state ###
  def checkPrecondition(self, pred):
    """
    Given this precondition and the current state, does this precondition pass?
    args:
    pred (Predicate)
    
    return:
    boolean - whether or not this event is valid
    """
    if pred.isPrecondition == False: return False #it's an effect, don't consider it

    ### Your code here ###
    #TODO: check pred against self.state
    # You can do direct matches or "close enough" (i.e., similarity) matches
    #e.g.:
    # print("state:",self.state)
    # print(self.curr_NER)
    a=nlpp(pred.statement)
    for item in self.state[self.curr_NER['PersonX']]:
      b=nlpp(item)
      if a.similarity(b)>=0.7:
        return True
    #####################

    return False

  def getPreconditions(self):
    """
    Given the input event string (self.curr_event), 
    return a list of preconditions (list of Predicate objects)
    """
    pre = []  

    ### Your code here ###
    #TODO: complete the list using relations from "all_relations"
    #an example:
    for rel in preq_relations:
      comet_out = callCOMET(self.curr_event,rel)
      for fact in comet_out:     
        filled = fillEntityTags(fact,self.curr_NER)
        pre.append(Predicate(rel, filled, True)) #Predicate(rel, statement, isPrecondition, negated)
    ######################

    return pre

  def checkValidity(self):
    """
    Goes through all the preconditions to check to see if
    this event can be added to the state.
    A precondition is considered valid as long as 
    
    return:
    boolean - whether or not this event is valid
    """
    # print(self.state)
    preconds = self.getPreconditions()
    valids = []
    # print("preconds: ",preconds)
    for precond in preconds:
      print(precond.statement)
      valid = self.checkPrecondition(precond)
      print(valid)
      if valid:
        valids.append(precond)
    if valids:
      self.preconditions += valids
      return True    
    return False


  ### Once validated, update the schema ###
  def getEffects(self):
    """
    Given the input event string (self.curr_event),
    return a list of effects (list of Predicate objects)
    """
    effects = []

    ### Your code here ###
    #TODO: complete the list using relations from "all_relations"
    #an example:
    for rel in effects_relations:
      fact = callCOMET(self.curr_event,rel)[0]
      filled = fillEntityTags(fact,self.curr_NER)
      effects.append(Predicate(rel, filled, False)) #Predicate(rel, statement, isPrecondition, negated)
    ######################

    return effects   


  def updateSchema(self, event, NER_dict):
    """
    Given an input event string (event), check the validity of adding it to the state,
    and update the schema state (self.state) with new effects
    args:
    event(str) - received from the parser; phrase from the original input sentence with PersonX/PersonY labels
    NER_dict(dict) - person names and their corresponding tags for a given subevent/phrase, e.g. NER_dict['PersonX'] = "Cindy"
    """
    self.curr_event = event
    self.curr_NER = NER_dict

    #1) check validity
    valid = self.checkValidity()
    print("Preconditions",self.preconditions[0].statement)

    #2) update the state
    effects = self.getEffects()
    print("Effects", effects[0].statement)


    ### Your code here ###

    #TODO: add these new effects to the state    
    for effect in effects:
      if effect.relation in effects_relations:
        self.state[NER_dict['PersonX']].add(effect.statement)

    # TODO: (optional) remove facts that aren't true anymore
    # Although this is ideal, it might be hard to figure out when facts are negated

    ######################





# What to Turn In
* Your code
  * Including your answers outputted for the Story Tracking Questions
* A document, including:
  * A brief description/diagram of the structure of your schema
  * Your answers to the questions about COMET-ATOMIC




## Story Tracking Questions
Run the following stories through your system and print out your schema after each sentence (or subevent if there are multiple events in a sentence).

For each scenario, report:
- the number of the scenario,
- the bit of code that runs the story, and
- keep the output of your schema on that story in your ipynb. It should be printing the schema at each timestep. (No fancy formatting is necessary. Just printing the dictionary is fine.)

Note: The first sentence of each story should initialize your schema state.

**Do not change your schema code in between running these examples! Your final schema code should be able to run multiple scenarios.**

In [17]:
stories = {
    1: ["Gina misplaced her phone.", "Gina looks for her phone in the living room.", "Gina remembers leaving her phone in the car.", "Gina goes back to the car.", "Gina finds her phone in the car."],
    2: ["Phil was at the community pool.","Phil thought he could go out to the deeper end by himself.","Phil jumps into the deep end.","Phil has trouble staying afloat.","The lifeguard had to help Phil out of the water."],
    3: ["Amy was happy her first class in junior high was all new kids.", "Amy introduced herself to the girl seated next to her.", "The girl was even more nervous than Amy to make friends.", "The girls talked and bonded over their love of books.", "The girls decided to meet up after school to go to the library."],
    4: ["Xander's dog hates his treats.", "Xander decided to go buy some new dog treats.", "None of the dog treats at the pet store looked tasty.", "Xander decided to buy his dog some salmon from the fish market.", "Xander's dog loved the salmon."],
    5: ["Tim has never cooked for his family.", "Tim decided to follow an old family recipe.", "Tim's grandma told him anybody could make the recipe.", "Tim made a whole meal for his family in one hour.", "Tim's family all loved the meal."]
}


In [19]:
# This is where everything is called
start = defaultdict(set)
start.update({"John": set(["John is hungry"])})

# Example call
schema = Schema(start)
sent = "John eats an apple."
s = SentParser(sent)
schema.updateSchema(s.new_phrase,s.entities)
print(schema.state)

John buys an apple
False
John eats an apple
True
John is hungry
True
apples
False
apple
False
John eats the apple
True
John takes a bite
True
John eats an apple
True
John is full
True
John is full
True
to be healthy
False
to satisfy hunger
False
to eat something
False
to eat
False
none
False
John puts the apple in their mouth
True
John takes a bite of the apple
True
John puts the apple in a bowl
True
John puts the apple in his mouth
True
John eats an apple
True
John goes to the store
True
John eats an apple
True
John buys an apple
False
John is hungry
True
none
False
they are full
False
they are hungry
True
none
False
satisfied
False
healthy
False
John eats the apple
True
John eats an apple
True
John takes a bite
True
John is full
True
none
False
John puts the apple in their mouth
True
John takes a bite of the apple
True
John puts the apple in a bowl
True
John puts the apple in his mouth
True
John puts the apple in the fridge
False
John puts the apple in their mouth
True
John buys an a

In [20]:
# Uncomment this when you're ready

# Running through the above stories
start = defaultdict(set)

### Your code here ###
# TODO: initialize your start state with the first sentence of the story
# (You can do this here or inside of the Schema object)
start['Gina'] = set(["Gina used her phone."])
######################

schema = Schema(start)
i=1
for sent in stories[i]:
  s = SentParser(sent)
  try:
    schema.updateSchema(s.new_phrase,s.entities)
    print(schema.state)
    print("story "+str(i)+" succeeds")
  except:
    print("story "+str(i)+" fails")

Gina looks for their phone
True
Gina looks for her phone
True
Gina loses their phone
True
Gina loses her phone
True
Gina lost their phone
True
to look for the phone
True
to look for their phone
True
to find their phone
True
to look for it
False
none
False
Gina loses their phone
True
Gina looks for their phone
True
Gina lost their phone
True
Gina loses her phone
True
Gina loses their phone
True
Gina looks for their phone
True
Gina looks for her phone
True
Gina looks for the phone
True
Gina loses their phone
True
Gina loses her phone
True
Gina looks for their phone
True
Gina loses their phone
True
Gina looks for her phone
True
Gina loses her phone
True
Gina lost their phone
True
Gina loses their phone
True
Gina lost their phone
True
Gina loses her phone
True
Gina lost her phone
True
none
False
Gina looks for the phone
True
Gina looks for their phone
True
Gina looks for her phone
True
Gina calls the police
True
none
False
Gina looks for their phone
True
Gina looks for her phone
True
Gina 

In [21]:
start = defaultdict(set)

### Your code here ###
# TODO: initialize your start state with the first sentence of the story
# (You can do this here or inside of the Schema object)
start['Phil'] = set(["Phil spent his free time."])
######################

schema = Schema(start)
i=2
for sent in stories[i]:
  s = SentParser(sent)
  try:
    schema.updateSchema(s.new_phrase,s.entities)
    print(schema.state)
    print("story "+str(i)+" succeeds")
  except:
    print("story "+str(i)+" fails")


start['Xander'] = set(["Xander had a new dog."])

Phil goes to the community pool
True
Phil went to the community pool
True
Phil swims in the pool
False
Phil went to the pool
True
Phil goes to the pool
True
go to the pool
False
Phil swims
False
Phil gets wet
False
to go swimming
False
to swim
False
Phil goes to the pool
True
Phil went to the pool
True
Phil goes swimming
False
Phil went swimming
False
none
False
Phil goes to the community pool
True
Phil swims in the pool
False
Phil went to the community pool
True
Phil goes to the pool
True
Phil swims in the water
False
Phil goes to the pool
True
Phil went to the pool
True
Phil goes swimming
False
Phil went swimming
False
none
False
Phil goes to the pool
True
Phil went to the pool
True
Phil went swimming
False
none
False
relaxed
False
Phil jumps in the pool
False
Phil jumps in the water
False
Phil jumps into the water
False
Phil swims
False
none
False
Phil goes to the community pool
True
Phil swims in the pool
False
Phil went to the community pool
True
Phil goes to the pool
True
Phil sw

In [22]:
start = defaultdict(set)

### Your code here ###
# TODO: initialize your start state with the first sentence of the story
# (You can do this here or inside of the Schema object)
start['Amy'] = set(["Amy was firstly in junior high school."])
######################

schema = Schema(start)
i=3
for sent in stories[i]:
  s = SentParser(sent)
  try:
    schema.updateSchema(s.new_phrase,s.entities)
    print(schema.state)
    print("story "+str(i)+" succeeds")
  except:
    print("story "+str(i)+" fails")

Amy goes to the new school
True
Amy goes to the new class
True
Amy looks at the new kids
True
Amy goes to school
True
Amy meets new friends
False
Amy goes to class
True
Amy learns a lot
False
Amy goes to school
True
none
False
Amy smiles
False
none
False
person x is happy
False
personx is happy
False
person x was happy
False
excited to learn
False
Amy goes to the new school
True
Amy goes to the new class
True
Amy meets new people
False
Amy meets new kids
False
Amy meets new friends
False
Amy goes to a new school
True
Amy goes to the new school
True
Amy meets new kids in school
True
Amy goes to school
True
none
False
Amy goes to the new school
True
Amy goes to the new class
True
Amy goes to school
True
Amy meets new kids
False
Amy meets new people
False
Amy goes to the new school
True
Amy goes to the new class
True
Amy meets new people
False
Amy meets new friends
False
Amy meets new kids
False
Amy goes to the new school
True
Amy goes to the new class
True
Amy meets new kids
False
Amy go

/usr/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__mai

False
friend
False
girl
True
none
True
to make friends
True
to be social
True
to be liked
True
to be friends
True
to talk to people
True
to talk to someone
True
to talk to them
True
to make a plan
True
to have a friend
True
to talk to Amy
True
to ask Amy out
True
to get to know them
True
to make friends
True
to be friends
True
Preconditions Amy goes to the new school
Effects Amy meets a new girl
defaultdict(<class 'set'>, {'Amy': {'Amy smiles at the girl', 'Amy gets nervous', 'Amy meets a new girl', 'Amy sees a girl sitting next to her', 'gets to know new people', 'Amy meets a new girl at school', 'to make friends', 'to be friendly', 'none', 'to talk to the girl', 'Amy goes to a new school', 'excited', 'to get to know them', 'friendly', 'happy', 'gets to know the girl', 'nervous', 'Amy makes new friends', 'Amy was firstly in junior high school.', 'Amy walks up to the girl'}})
story 3 succeeds
book store
story 3 fails
library
story 3 fails


In [23]:
start = defaultdict(set)

### Your code here ###
# TODO: initialize your start state with the first sentence of the story
# (You can do this here or inside of the Schema object)
start['Xander'] = set(["Xander had a new dog."])
######################

schema = Schema(start)
i=4
for sent in stories[i]:
  s = SentParser(sent)
  try:
    schema.updateSchema(s.new_phrase,s.entities)
    print(schema.state)
    print("story "+str(i)+" succeeds")
  except:
    print("story "+str(i)+" fails")

Xander feeds the dog dog treats
True
Xander feeds the dog dog food
True
Xander buys dog treats
False
Xander feeds the dog treats
True
Xander buys dog food
False
to feed the dog
True
to buy more treats
False
to buy dog treats
True
to buy dog food
True
eat the treats
False
Xander feeds the dog dog treats
True
Xander feeds the dog dog food
True
Xander feeds the dog too much
True
Xander feeds the dog a treat
True
Xander gives the dog treats
True
Xander feeds the dog dog treats
True
Xander gives the dog a treat
True
Xander feeds the dog dog food
True
Xander feeds the dog a treat
True
Xander buys dog treats
False
Xander gives the dog a treat
True
Xander feeds the dog dog treats
True
Xander feeds the dog dog food
True
Xander feeds the dog a treat
True
Xander gives the dog treats
True
personx's dog hates the treats
True
person x's dog hates the treats
True
person x's dog hates person x
False
personx's dog hates personx
False
none
False
Xander takes the dog to the vet
True
Xander gives the dog 

In [24]:
start = defaultdict(set)

### Your code here ###
# TODO: initialize your start state with the first sentence of the story
# (You can do this here or inside of the Schema object)
start['Tim'] = set(["Tim wanted to cook for his family"])
######################

schema = Schema(start)
i=5
for sent in stories[i]:
  s = SentParser(sent)
  try:
    schema.updateSchema(s.new_phrase,s.entities)
    print(schema.state)
    print("story "+str(i)+" succeeds")
  except:
    print("story "+str(i)+" fails")

Tim goes to the grocery store
True
Tim buys a new cookbook
True
Tim buys a new stove
True
Tim buys a new kitchen
True
Tim cooks dinner
False
to learn how to cook
True
to learn to cook
True
Tim cooks a meal
True
Tim cooks dinner
False
to cook
True
Tim wants to be a chef
True
Tim wants to have a family
True
Tim buys a new stove
True
Tim buys a new kitchen
True
none
False
Tim goes to the grocery store
True
Tim buys a new cookbook
True
Tim goes to a new restaurant
True
Tim buys a new stove
True
Tim buys a new kitchen
True
Tim goes to the grocery store
True
Tim buys a new cookbook
True
Tim goes to a new restaurant
True
Tim buys a new stove
True
Tim buys a new kitchen
True
Tim wants to be a chef
True
Tim wants to have a family
True
Tim buys a new stove
True
Tim buys a new kitchen
True
none
False
Tim buys a new stove
True
Tim cooks for their family
True
Tim buys a new kitchen
True
Tim cooks for PersonY
True
Tim cooks a meal
True
Tim goes to the grocery store
True
Tim buys a new cookbook
True


/usr/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__mai

False
bread
False
food
False
to be a good cook
True
to make a good meal
True
to follow a family recipe
True
to follow the family recipe
True
to follow a family tradition
True
to read the recipe
True
to learn the recipe
True
to know the recipe
True
to have a recipe
True
none
True
none
True
to eat the recipe
True
to eat the food
True
to help Tim
True
to eat the meal
True
Preconditions Tim goes to the grocery store
Effects Tim wants to try a new recipe
defaultdict(<class 'set'>, {'Tim': {'Tim buys a new cookbook', 'Tim wanted to cook for his family', 'to eat the recipe', 'satisfied', 'lazy', 'to be independent', 'Tim wants to try a new recipe', 'gets yelled at by family', "Tim's family is hungry", 'to be a good cook', 'none', 'determined', "Tim doesn't know how to cook", 'Tim makes a new recipe', 'to learn how to cook', 'Tim wants to make a new recipe', 'Tim eats the recipe', 'ashamed'}})
story 5 succeeds
Tim's grandma taught him how to cook
True
Tim's grandma gives Tim a recipe
True
Tim'

## Questions about COMET-ATOMIC



1.   What types of stories is COMET-ATOMIC *good* at tracking? In other words, what types of information is it good at modeling? (It might help to think about how COMET-ATOMIC compares to other knowledge bases.)
2.   What types of stories is COMET-ATOMIC *bad* at tracking?

   2a. Do you think any of the other knowledge bases mentioned in class could better model these?



# Grading Rubric
This assignment is 10% of your final grade, broken down as the following for grading:
*   Code - 2 points
*   Schema Diagram/Explanation - 2 points
*   Story Tracking Questions - 15 points
*   COMET-ATOMIC Questions - 6 points

Total: 20 points
